In [1]:
# This notebook shouldn't be used, copy the queries to MotherDuck UI to show the UI!
import duckdb
conn = duckdb.connect('md:')


In [ ]:
conn.sql('ATTACH md:_share/duckdb_stats/d87758b0-7806-4803-8c5d-17144f56bd12')


In [ ]:
conn.sql('DROP DATABASE IF EXISTS my_demo_share CASCADE')
conn.sql('CREATE DATABASE my_demo_share')


In [ ]:
conn.sql("""
CREATE TABLE my_demo_share.weekly_download AS
SELECT 
    EXTRACT(year FROM timestamp) AS year, 
    EXTRACT(month FROM timestamp) AS month, 
    COUNT(*) AS download_count 
FROM 
    duckdb_stats.pypi 
GROUP BY 
    year, month
order by 
    year DESC, month DESC;
""")


In [ ]:
conn.sql("""
CREATE TABLE my_demo_share.country_download AS
SELECT 
    country_code, 
    COUNT(*) as download_count 
FROM 
    duckdb_stats.pypi
GROUP BY 
    country_code 
ORDER BY 
    download_count DESC 
LIMIT 1000;
""")


In [ ]:
conn.sql("""
CREATE TABLE my_demo_share.weekly_duckdb_version AS
SELECT 
    EXTRACT(year FROM timestamp) AS year,
    EXTRACT(week FROM timestamp) AS week,
    file->>'version' AS duckdb_version,
    COUNT(*) AS download_count
FROM 
    duckdb_stats.pypi
WHERE 
    project = 'duckdb'
GROUP BY 
    year, week, duckdb_version
ORDER BY 
    year DESC, week DESC, download_count DESC;
""")


In [ ]:
conn.sql("""
CREATE table my_demo_share.weekly_python_version AS
SELECT 
    EXTRACT(year FROM timestamp) AS year, 
    EXTRACT(week FROM timestamp) AS week, 
    CONCAT(SPLIT_PART(python_version, '.', 1), '.', SPLIT_PART(python_version, '.', 2)) AS python_major_minor_version,
    COUNT(*) AS download_count
FROM (
    SELECT timestamp, 
           details->>'python' as python_version 
    FROM duckdb_stats.pypi
    WHERE (details->>'python' IS NOT NULL) AND (details->>'python' <> '')
) as filtered_data
GROUP BY 
    year, week, python_major_minor_version
ORDER BY 
    year DESC, week DESC, download_count DESC;

""")
